In [1]:
##
# imports
##

import random # generating pseudo-random, reproducable cities
from math import sqrt, ceil, inf # calculating distances between cities, separating individuals into groups, results
from copy import deepcopy # populating generation
from enum import Enum, StrEnum # type defs
from datetime import datetime # randomization

##
# constants 
##

ALPHABET='abcdefghijklmnopqrstuvwxyz'
CITY = 0
ORDER = 1
PLANE_DIMENSION_X = 200 
PLANE_DIMENSION_Y = 200

##
# enums
##

Selection = Enum('Selection', ['Rank','Tournament','Elitism', 'Roulette_Wheel'])
Crossover = Enum('Crossover', ['One_Point', 'Two_Point', 'Uniform'])
Mutation = Enum('Mutation', ['Boundary', 'Aritmetic', 'Order'])
Repopulation = Enum('Repopulation', ['Steady_State', 'Generational'])
Stop = Enum('Stop', ['Generations', 'Threshold'])

class Error(StrEnum):
    Invalid_Selection = "Error - No valid selection method passed."
    Invalid_Mutation = "Error - No valid mutation method passed."
    Invalid_Crossover = "Error - No valid crossover method passed."
    Invalid_Repopulation = "Error - No valid repopulation method passed."
    Invalid_Stop = "Error - No valid stopping condition passed."

### Representing Initial Problem Data

__Initial Thoughts__

Here, the representation of each city is relatively simple. Each city only requires a set of location coordinates (x,y) which are contained within the class, `City`, and assigned during instantiation of a new `City` object. The location coordinates should also be non-mutable as no change would sensibly be necessary. The more pressing concern is generating the distance between every other city in the list. 

__Calculating Distances__

While it would be interesting to attempt the computation during the intial creation of the list, this approach would not be feasible because the addition of new cities would require a separate record of computations introducing the possibility of incomplete distance data. Instead, each `City` object can compute its distance from every other city after all cities have been instantiated. This approach is best suited as a method of the `City` class and would require the complete list of cities in the problem space.

It is also assumed that there are no objects, except other cities, blocking the most direct route from one city to another. For this reason, the pythagorean theorem is used to calculate the shortest path (straight line) between two cities. 

__Additional Considerations__ 

With the required attributes and methods defined, it's easy to see there's still a glaring lack of organization – more specifically, identification. While iterating over a list of non-identifiable `City` objects would work, it's not very readable. There's also performance implications of using a list (i.e. O(n) search). To fix this, each city object will be given an unique alphanumeric identifier. This minor change allows the use of the time-efficient dictionary data structure to store and access each `City` object. The same reasoning extends to storing the distance from a city to all other cities; this data will stored inside each `City` object as an attribute. 

Lastly, making public data within an object accesible with as little external computation as possible is one of the standards in OOP. Various "getters and setters" will be used to satisfy this standard.

_Notes_:

- As required by the assignment, test runs should be completed with at least 25 different locations generated at random. 
- Creating a reusable function that generates and returns a list of random locations based on a given seed ensures randomness and reproducibility. 
- The distance between two cities is rounded 3 decimal points to ensure conversion to whole integers in later computations. Floating point errors suck.


In [2]:
class City:
    
    def __init__(self, x:int, y:int, identifier:str):
        """Initialize object with attributes.
        
        Args: 
            x : the x coordinate of the city location.
            y : the y coordinate of the city location.
            identifier : a unique id for the city
        """
        self._x = x
        self._y = y
        self._id = identifier
        self._distance_to_other_cities = {}
        
    ##
    # read-only attributes
    ##
    
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y
    
    @property
    def id(self):
        return self._id
    
    ##
    # general utilities
    ##
    
    def __str__(self):
        """List city id and distance to all other cities. 
        """
        string = f"City : {self._id}\nDistance to other cities:\n"
        
        for key in self._distance_to_other_cities.keys():
            string+=f"\t{key} : {self._distance_to_other_cities[key]} km\n"
            
        return string
            
        
    def calculate_distance_to_other_cities(self, cities:dict[str, list['City', int]]):
        """Calculate and store the distance to all other cities.
        
        Args:
            cities : a list of all City objects and their order. Can contain current city.
        """
        for key in cities.keys():
            
            # skip current city if in dictionary.
            if (key == self._id):
                continue
            
            other_city = cities[key][CITY]
            
            # skip calc if distance has already been computed by other city.
            if ((distance := other_city.get_distance_to_city(self._id)) != None):
                self._distance_to_other_cities[key] = distance
                continue
                
            # calculate absolute value of difference in x,y coords.
            distance_x = abs(self._x - other_city.x)
            distance_y = abs(self._y - other_city.y)
            
            # apply pythagorean theorem to find shortest-path distance. Round to nearest thousandth.
            direct_distance = round(sqrt( distance_x**2 + distance_y**2), 3)
            
            # add to distance dict.
            self._distance_to_other_cities[key] = direct_distance
            
    def get_distance_to_city(self, other_city:str) -> float:
        """Retrieve distance to other city if available.
        Args:
            other_city : the id of the city to use when searching for a distance.
        Returns:
            A float representing the distance between the current city and passed city in km if available; 
            None if not.
        """
        return self._distance_to_other_cities.get(other_city, None)
    
            

def generate_random_cities(seed:str, n:int=25) -> dict[str, list[City, int]]:
    """Generate N cities with pseudo-random locations within plane.
    
    Args:
        seed : the seed used to acheive psuedo-randomization.
        n : the desired number of cities to be generated.
            Defaults to 25.
    Returns:
        A dictionary containing N entries with city identifier as key. The value is set to a list containing the 
        City object and its order(set to -1 by default).
    """
    cities = {}
    
    # set seed before running randomizer.
    random.seed(seed)
    
    for i in range(n):
        # generate new id of the form {letter}{number}. Loops alphabet with incrementing numbers.
        new_id = f"{ALPHABET[i % len(ALPHABET)]}{i//len(ALPHABET)}"
        # generate pseudo-random coordinates between 0 and plane_dimension_x/y
        rand_x = int(random.random()*PLANE_DIMENSION_X)
        rand_y = int(random.random()*PLANE_DIMENSION_Y)
        
        # create new city and add to dictionary.
        cities[new_id]= [City(x=rand_x, y=rand_y, identifier=new_id), -1]
        
    # calculate the distance between all pairs of cities.
    for city in [cities[city_id][CITY] for city_id in cities]:
        city.calculate_distance_to_other_cities(cities)
    
    # ensure remaining random generation is as random as possible.
    random.seed(datetime.now().timestamp())
    return cities
    

### Encoding Solution

__Decision__

The TSP focuses on optimization and ordering of elements which means the preferred choice of encoding is _order encoding_.

__Wait A Second...__

You may have noticed that a value of -1 was attached to all `City` objects during the `generate_random_cities` function; this is because the the data that would be stored in a separate order-encoded data structure is the exact same as the data used everywhere else to access each city. Instead of duplicating this data, it made sense to attach the ordering to the city id as well. This may lead you to believe the encoding of the solution can be considered hybrid with elements of both _order_ and _real-value encoding_; not exactly.

In order for the encoding to be considered _real-value_, the real values stored must actively be examined as a factor for determining the fitness(more on this later) or viability of the solution. While the real values, the city identifier and reference to the object in this case, are a handy utility during the computation process, they are not a factor that affect the actual computations. For this reason, this implementation should only be considered _order encoded_.

__Additional Considerations__

The non-contiguous nature of dictionaries will introduce extra complexity while calculating the fitness score (at most O(n)), however, the trade-off is well worth the extra complexity.

Also, encoding the data as a tree may be suitable here as well, however, tree operations are generally more complex than operations on a data structure like a dict or array which may increase algorithm runtime without any real benefits.

_Notes_:
- Since ordering schemes typically never include negative positions, any city that has received a valid position will not have the value -1.


### Calculating Fitness of Individuals

It's important to discuss how the fitness of an individual is determined before moving forward; after all, this attribute will be the major factor in the rest of the implementation.

__Disqualifying Factors__

First, we already know that each city must be visited exactly once from the problem description. Therefore, any solution that does not have a non-negative order for every city in plane should be penalized in such a way to minimize the chance reproduction; in this case, we'll set the fitness score of the solution to 0 (more on this later). 

Next, we know that an ordering scheme requires _unique, contiguous_ symbols to represent the sequence in which all actions should take place. This property implies that a valid solution will only contain cities with a unique order number ranging from the a non-negative(0) to N-1 in this implementation. Any solution that violates this property should also be categorizes as unfit for reproduction(fitness score = 0).

__Ranking Factors__

With those two disqualifying conditions set, there's only one factor remaining to determine the fitness score of a solution : the total distance of the path. Since the optimal solution should be the solution with the least total distance travelled, solutions with smaller total travel distances should receive better scores than solutions with larger total travel distances. 

__Additional Considerations__

In an effort to minimize floating point and other arithmetic errors, the fitness score function will always return a non-negative integer.

I've taken the liberty of defining some utilities here as well in the DRY spirit.

_Notes_:
- "Individual" and "solution" can be used interchangeably here. "Solution" made more sense to use within the context of the explanation.
    - Also, since we can start thinking of solutions as individuals from this point forward and typing out `dict[str, list[City, int]]` is tiring, we'll define a pretty handy alias `Individual` here.
- The term "better" is also used loosely here. In the context of this implementation, a "better" fitness score could be one that is lower than another. For the sake of simplicity (and my brain), solutions resulting in lower travel distances will receive a higher fitness score.
    - In order to implement this properly, a small amount of reasoning is required:
        1. The worst possible solution has a path where every edge is between two cities exactly PLANE_DIMENSION_X, PLANE_DIMENSION_Y (maximum plane size) units away from each other. These solutions will have a maximum total distance of `Number of cities + 1 (final city -> origin city) * √ (PLANE_DIMENSION_X^2 + PLANE_DIMENSION_Y^2)`. 
        2. Direct distances are rounded to the nearest thousandth, so multiplying by 1,000 will remove any decimal values. 
   - Therefore, to ensure lower distances receive higher scores and fitness scores are non-negative whole integers, the total distance will be used within a subtract operation a value roughly 100,000 times more than the maximum total distance. 

In [3]:
##
# aliases
## 
Individual = dict[str, list[City, int]]

def calculate_path_distance(individual: Individual) -> float:
    """Calculate the total travel distance of a solution.
    
    Args:
        individual : the representation of a potential solution.
    Returns : 
        A float rounded to 3 decimal places representing the total travel distance
        of the solution based on city order. 
    Note:
        A negative distance will be returned if all cities are not visited at least once
        with a unique ordering.
    """
    
    # define unfilled travel order as sequence of N insignificant strings.
    travel_order = ["Empty" for _ in range(len(individual))]
    
    # determine actual travel order of solution.
    for city_id in individual.keys():
        
        entry = individual[city_id]
        
        # handle case : non-whole integer order.
        if( type(entry[ORDER]) != int ):
            return -1.00
        
        # handle case : all cities not visited exactly once.
        if( (not 0 <= entry[ORDER] < len(individual)) or (travel_order[entry[ORDER]] != "Empty")):
            return -1.00
        
        # set city in correct travel order
        travel_order[entry[ORDER]] = entry[CITY]
    
    # calculate total travel distance of solution. Include distance from last city back to origin city.
    travel_distance = 0
    for i in range(len(travel_order)):
        curr_city = travel_order[i]
        prev_city = travel_order[i-1]
        travel_distance += curr_city.get_distance_to_city(prev_city.id)
    
    return round(travel_distance, 3)
        

def calculate_fitness(individual:Individual) -> int:
    """Calculate the fitness level of a solution.
    
    Args:
        individual : the representation of a potential solution.
    Returns : 
        A fitness score for the passed individual. 
    Note:
        A fitness score of 0 indicates individual does not meet the minimum requirements
        to be considered fit.
    """
    
    max_direct_distance = (len(individual)+1)*round(sqrt((PLANE_DIMENSION_X**2)+(PLANE_DIMENSION_Y**2)), 3)
    
    travel_distance = calculate_path_distance(individual)
    if (travel_distance < 0):
        return 0
    
    # convert distance to integer and return.
    return int( (max_direct_distance*(10**5)) - (travel_distance*1000))

def calculate_fitness_multi(individuals:list[Individual])-> list[dict[str,int]]:
    """Get fitness score of all individuals in a list.
    
    Args:
        individuals : a list of individuals to get the fitness score of.
    Returns:
        A list of dictionaries containing the index of each individual and their respective 
        fitness score in ascending order.
        Dictionary Entries:
            key : "individual_index" , value = index of individual list
            key : "score", value = fitness score of individual
    """
    fitness_scores = []

    # calculate and store fitness scores for each individual.
    for i, individual in enumerate(individuals):
        fitness_scores.append({"individual_index" : i, "score": calculate_fitness(individual)})

    # rank fitness scores in ascending order ("better" == higher fitness score == higher index in list).
    fitness_scores.sort(key=lambda x:x["score"])

    return fitness_scores

### Creating Initial Population

Creating the initial population is relatively simple now that the function for determining fitness is implemented. The steps are as follows:
1. Randomize the order of each city.
2. Run the randomized path through the `calculate_fitness()` function.
    1. Reject any individual with a fitness value of 0. Should not occur (read `Note` section).
    2. Run steps 1 & 2 until an individual with a fitness value > 0 is found.
3. Add individual to population.
4. Repeat until population reaches desired size.

__Additional Considerations__

To keep up with OOP principles, the population should be represented as a class, `Population`. Not much is needed at this point in the implementation – only attributes to hold the max population size, individuals in the population, and a method(s) to add individuals to the population. However, additional information and methods such as a generation counter and a function for creating a new generation will be useful during later computations. These additions will be discussed as they are added.

_Notes_:

- To prevent unnecessary repetitions of the randomization logic, the order number assignable to each city is restricted to a set of valid remaining numbers.

In [4]:
class Population:
    
    def __init__(self, capacity:int):
        """Initialize object with attributes.
        
        Args: 
            capacity : the capacity of the population.
        """
        self._capacity = capacity
        self._individuals = []
        self._generation = 0
        
    ##
    # read-only attributes
    ##
    @property
    def capacity(self) -> int:
        return self._capacity
    @property
    def individuals(self) -> list[Individual]:
        return self._individuals
    @property
    def current_generation(self) -> int:
        return self._generation
        
    ##
    # population utilities
    ##

    def add_individual(self, individual:Individual):
        """Add an individual to the population.

        Args:
            individual : the representation of a potential solution to add.
        """
        # reject individual if population at max capacity.
        curr_size = len(self._individuals)
        if (curr_size == self._capacity ):
            return

        # add individual at the end of list.
        self._individuals.append(individual)

    ##
    # selection utilities
    ## 

    def select_parents(self, method: Selection, to_select:int = 2) -> list[Individual] :
        """Selects individuals for reproduction.
        Args:
            method : the selection method to use.
                     Valid options are [Rank, Tournament, Elitism, Roulette_Wheel]
            to_select : how many individuals should be selected for reproduction.
                        Defaults to 2.
        Returns:
            A list of Individuals selected for reproduction based on specified method.
        """
        if (to_select == 0):
            return []
        
        fs = calculate_fitness_multi(self._individuals)

        if (method == Selection.Rank):
            return self._select_parents_rank_selection(to_select, fs)
        elif (method == Selection.Tournament):
            return self._select_parents_tournament_selection(to_select, fs)
        elif (method == Selection.Elitism):
            return self._select_parents_elitism_selection(to_select, fs)
        elif (method == Selection.Roulette_Wheel):
            return self._select_parents_roulette_wheel(to_select, fs)
        else:
            print(Error.Invalid_Selection)

    def _select_parents_rank_selection(self, to_select:int, fs: list[dict[str,int]]
                                ) -> list[Individual]:
        """Select parents based on rank of fitness score.

        Args:
            to_select : how many individuals should be selected for reproduction.
            fs : a list of all individuals in the population and their respective fitness scores.
        Returns:
            A list of Individuals selected for reproduction based on rank selection method.
        """
        wheel = []

        # assign slices of "wheel" based on individual's rank. ( higher index in list == more slices in "wheel").
        for i, entry in enumerate(fs):
            slices = i+1
            wheel.extend([entry["individual_index"]]*slices)

        # select {to_select} individuals at random to reproduce.
        parents = []
        for _ in range(to_select):
            parents.append(self._individuals[wheel[random.randint(0, (len(wheel)-1))]])

        return parents

    def _select_parents_tournament_selection(self, to_select:int, fs: list[dict[str,int]]
                                      ) -> list[Individual]:
        """Select parents based on rank of fitness score across random groups.

        Args:
            to_select : how many individuals should be selected for reproduction.
            fs : a list of all individuals in the population and their respective fitness scores.
        Returns:
            A list of Individuals selected for reproduction based on tournament selection method.
        """

        # assign individuals to groups at random. 
        groups = [[] for _ in range(to_select)]
        remaining_individuals = [i for i in range(len(self._individuals))]
        rep = 0
        group_size = 6
        
        # randomly choose {group size} individuals for each group.
        while(len(groups[-1])!= group_size):
            
            rand_ind = 0
            
            while(True):
                rand_int = remaining_individuals[random.randint(0, (len(remaining_individuals) - 1))]
                
                # 1 word : diversity.
                if (rand_int not in groups[rep % to_select]):
                    break
                
            groups[rep % to_select].append(rand_ind)

            rep += 1

        # select {to_select} individuals at random to reproduce.
        parents = []
        for group in groups:

            # get best fitness score in group.
            best_score = max(group) 

            # get individual's index.
            individual_index = fs[best_score]["individual_index"]

            # add individual to list of parents.
            parents.append(self._individuals[individual_index])

        return parents
    
    def _select_parents_elitism_selection(self, to_select:int, fs: list[dict[str,int]]
                                      ) -> list[Individual]:
        """Select parents based on fitness score.

        Args:
            to_select : how many individuals should be selected for reproduction.
            fs : a list of all individuals in the population and their respective fitness scores.
        Returns:
            A list of Individuals selected for reproduction based on elitism selection method.
        """
        # get top {to_select} individuals with best fitness scores.
        parents = []

        while (to_select != 0):
            # get {0 < to_select <= individuals in population} individuals in descending 
            # order of fitness score (higher fitness score == "better").
            elites = [self._individuals[entry["individual_index"]] for entry in
                           fs[: -(to_select+1):-1]]

            # handle condition : to_select > individuals in population.
            to_select -= len(elites)

            # add individuals to list of parents.
            parents.extend(elites)
        return parents

    def _select_parents_roulette_wheel(self, to_select:int, fs: list[dict[str,int]]
                                      ) -> list[Individual]:
        """Select parents based on fitness score.

        Args:
            to_select : how many individuals should be selected for reproduction.
            fs : a list of all individuals in the population and their respective fitness scores.
        Returns:
            A list of Individuals selected for reproduction based on roulette-wheel selection method.
        """
        wheel = []

        # get total fitness across all individuals in population.
        total_fitness = sum(entry["score"] for entry in fs)
        running_percentage = 0.00

        # determine slice size for each individual.
        for entry in fs:

            # get individual's probabilty of being selected.
            probability_selected = float(entry["score"]) / total_fitness
            wheel.append([entry["individual_index"], running_percentage, 
                          running_percentage+probability_selected])
            running_percentage+= probability_selected

        # select {to_select} individuals at random to reproduce.
        parents = []
        for _ in range(to_select):

            # get random number between 0-1.00
            random_slice = random.random()

            # find and add individual whose probability of selection matches random number.
            parents.extend([self._individuals[s[0]] for s in wheel if s[1] < random_slice <= s[2]])
        return parents

    ##
    # reproduction utilities
    ##

    def crossover_parents(self, method: Crossover, parents:list[Individual]) -> list[Individual]:
        """Crossover a list of Individuals.

        Args:
            method : the crossover method to use.
                     Valid options are [One_Point, Two_Point, Uniform]
            parents: a list of Individuals selected for crossover.
        Returns:
            A list of Individuals crossed using passed method.
        """
        if(method == Crossover.One_Point):
            return self._crossover_parents_one_point
        elif(method == Crossover.Two_Point):
            return self._crossover_parents_two_point
        elif(method == Crossover.Uniform):
            return self._crossover_parents_uniform
        else:
            print(Error.Invalid_Crossover)

    # TODO
    def _crossover_parents_one_point(self, parents:list[Individual]) -> list[Individual]:
        pass
    # TODO
    def _crossover_parents_two_point(self, parents:list[Individual]) -> list[Individual]:
        pass
    # TODO
    def _crossover_parents_uniform(self, parents:list[Individual]) -> list[Individual]:
        pass

    def mutate_parents(self, method: Mutation, parents:list[Individual]) -> list[Individual] :
        """Mutate a list of Individuals.

        Args:
            method : the mutation method to use.
                     Valid options are [Boundary, Arithmetic, Order]
            parents: a list of Individuals selected for mutation.
        Returns:
            A list of Individuals mutated using passed method.
        """
        if(method == Mutation.Boundary):
            return self._mutate_parents_boundary(parents)
        elif(method == Mutation.Aritmetic):
            return self._mutate_parents_arithmetic(parents)
        elif(method == Mutation.Order):
            return self._mutate_parents_order(parents)
        else:
            print(Error.Invalid_Mutation)

    # TODO
    def _mutate_parents_boundary(self, parents:list[Individual]) -> list[Individual]:
        pass
    # TODO
    def _mutate_parents_arithmetic(self, parents:list[Individual]) -> list[Individual]:
        pass

    def _mutate_parents_order(self, parents:list[Individual]) -> list[Individual]:
        """Mutate a list of Individuals using order mutation.

        Args:
            parents: a list of Individuals selected for mutation.
        Returns:
            A list of Individuals with slightly mutated travel paths.
        """
        children = []
        for individual in parents:
            
            # avoid altering Individual object stored in current population.
            individual = deepcopy(individual)
            
            # get two cities in each individual to swap.
            generate_random_pair = lambda : (random.randint(0, len(individual) -1 ), random.randint(0, len(individual)-1))
            mutation_indices = ()
            while (True):
                mutation_indices = generate_random_pair()
                
                # ensure each cities are unique.
                if (mutation_indices[0] != mutation_indices[1]):
                    break
            # swap traversal order of cities.
            city_ids = list(individual.keys())
            city = lambda index : individual[city_ids[mutation_indices[index]]]
            city(0)[ORDER], city(1)[ORDER] = city(1)[ORDER], city(0)[ORDER]
            children.append(individual)
            
        return children

    ##
    # repopulation utilities
    ##

    def populate_next_generation(self, method:Repopulation, replacements:list[Individual]):
        """Create new generation of individuals based on passed method.

        Args:
            method : the repopulation method to use.
                     Valid options are [Steady_State, Generational]
            replacements: a list of Individuals that may be used to replace those 
                          in current population.
        """
        if(method == Repopulation.Steady_State):
            self._populate_next_generation_steady_state(replacements)
        elif(method == Repopulation.Generational):
            self._populate_next_generation_complete(replacements)
        else:
            print(Error.Invalid_Repopulation)
            return
        
        self._generation += 1

    def _populate_next_generation_steady_state(self, replacements:list[Individual]):
        """Create new generation of individuals using steady-state method.

        Args:
            replacements: a list of Individuals that may be used to replace those 
                          in current population.
        """
        # determine fitness scores for all individuals considered for next generation.
        fs = {}
        fs["c_pop"] = calculate_fitness_multi(self._individuals)
        fs["rpm"] = calculate_fitness_multi(replacements)

        # determine fittest individuals and place them in the next generation.
        new_generation = []
        remaining_cap = self._capacity
        c_pop_counter = len(fs["c_pop"]) - 1
        rpm_counter = len(fs["rpm"]) - 1
        while(remaining_cap != 0):
            # handle condition : no more individuals available for comparison in replacements.
            if (rpm_counter < 0):
                new_generation.extend([self._individuals[ind["individual_index"]] for ind in 
                                       fs["c_pop"][c_pop_counter - remaining_cap+1 : c_pop_counter +1]])
                break

            # handle condition : no more individuals available for comparison in current population. Unlikely.
            if(c_pop_counter < 0):
                new_generation.extend([replacements[rpm["individual_index"]] for rpm in 
                                       fs["rpm"][rpm_counter - remaining_cap+1 : rpm_counter +1]])
                break

            c_pop_entry = fs["c_pop"][c_pop_counter]
            rpm_entry = fs["rpm"][rpm_counter]

            # fill new generation with fittest individuals.
            if (c_pop_entry["score"] > rpm_entry["score"]):
                new_generation.append(self._individuals[c_pop_entry["individual_index"]])
                c_pop_counter -= 1
            else:
                new_generation.append(replacements[rpm_entry["individual_index"]])
                rpm_counter -= 1

            remaining_cap -= 1

        # set new population.
        self._individuals = new_generation


    def _populate_next_generation_complete(self, replacements:list[Individual]):
        """Create new generation of individuals using generational method.

        Args:
            replacements: a list of Individuals to replace those in current population.
        """
        # replace all individuals in current population.
        if (len(replacements) < self._capacity):
            self._individuals = [replacements[i % len(replacements)] for i in range(self._capacity)]
        else:
            self._individuals = replacements[:self._capacity]

    ##
    # final solution utilities
    ##

    def found_solution(self, method:Stop, *condition) -> tuple[bool, Individual]:
        """Determine if solution can be selected based on stopping condition.

        Args:
            method : the stopping condition to use.
                     Valid options are [Generations, Threshold]
            *condition : any extra arguments not specified such as threshold value or
                          required generations.
        Returns:
            A tuple containing a boolean that represents whether the stopping condition
            was met and a Individual that is the most optimal solution.

        Note:
            If the stopping condition is not met, the return value will be [False, None].
        """
        if(method == Stop.Threshold):
            return self._found_solution_fitness_threshold(condition[0])
        elif(method == Stop.Generations):
            return self._found_solution_fixed_generations(condition[0])
        else:
            print(Error.Invalid_Stop)

    def _found_solution_fitness_threshold(self, threshold:int) -> list[bool, Individual, int]:
        """Determine if solution can be selected based on combined fitness of population.

        Args:
            threshold : the lowest combined fitness value of all Individuals before a 
                        solution can be selected.
        Returns:
            A list containing a boolean that represents whether the stopping condition
            was met, the Individual with the highest fitness score, and the combined fitness
            score of all individuals in the population.

        Note:
            If the combined fitness value of all Individuals in the population is not >= threshold,
            the return value will be [False, None, None].
        """
        fs = calculate_fitness_multi(self._individuals)

        # reject if stopping condition not met.
        total_fitness = sum( entry["score"] for entry in fs )
        if (total_fitness < threshold):
            return [False, None, None]
        else:
            return [True, self._individuals[fs[-1]["individual_index"]], total_fitness]
        
    def _found_solution_fixed_generations(self, max_generations:int) -> list[bool, Individual, int]:
        """Determine if solution can be selected based on the fixed generations stopping condition.

        Args:
            generations : the total number of generations that need to pass before a 
                      solution can be selected.
        Returns:
            A list containing a boolean that represents whether the stopping condition
            was met, the Individual with the highest fitness score, and the combined fitness
            score of all individuals in the population.

        Note:
            If the number of generations passed is not >= max generations, the return 
            value will be [False, None, None].
        """
        # reject if stopping condition not met.
        if (self._generation != max_generations):
            return [False, None, None]
        # find best solution (Individual in population with highest fitness score).
        else:
            fs = calculate_fitness_multi(self._individuals)
            total_fitness = sum(entry["score"] for entry in fs)
            return [True, self._individuals[fs[-1]["individual_index"]], total_fitness]
            
                

def generate_initial_population(capacity:int, cities:dict[str, list[City, int]]) -> Population :
    """Generate an initial population of individuals (potential solutions).
    
    Args:
        capacity : the desired size of the population.
        cities : a list of all City objects and their order.
    Note:
        City order is not taken into account until after randomization.
    """ 
    initial_population = Population(capacity)
    
    # assign random order to each city for {size} individuals. Add indivduals to population.
    for _ in range(capacity):
        individual = deepcopy(cities)
        while(True):
            remaining_positions = [i for i in range(len(individual))]
            for city_id in individual.keys():
                # randomly choose from remaining pool of numbers and assign to city.
                rand_num = remaining_positions[random.randint(0, (len(remaining_positions) - 1))]
                individual[city_id][ORDER] = rand_num

                # remove number from remaining pool.
                remaining_positions.remove(rand_num)

            # verify solution is valid
            if(calculate_fitness(individual) != 0):
                break
        
        # add individual to population.
        initial_population.add_individual(individual)
        
    return initial_population

### Selecting Parents

The best choices of parent selection in this implentation are _rank_ and _tournament selection_. 

__Explanation__

The major reason for this is simplicity. _Roulette-wheel selection_ involves using an individual's fitness score in some division-focused expression to determine the individual's likeliness of being selection for reproduction. This reintroduces the possibility of floating point and other arithmetic errors back into the solution. 

_Rank_ and _tournament selection_ do not have this drawback, though they do decrease the chance of diversity among the population. Along the same train of thought, _elitism selection_ leaves much to be desired and doesn't provide any significant benefit over the other methods. 

__Final Decision__

To create as diverse a population as possible within the context of the implementation, _rank selection_ will be used as the primary selection method.  

Aside from electing a selection method, there's the question of where it should be implemented – and the answer is also straightforward. The selection method will require access to all of the data stored within a `Population` object, therefore, it is implemented as a method within the `Population` class.

_Notes_:
- Though _rank selection_ is the primary selection method used, _tournament_ , _elitism_ , and _roulette-wheel selection_ have been implemented (as time allows) for testing and comparison purposes.
- __Individuals with a fitness level of 0 are not explicitly excluded from reproduction.__ Good luck underdogs, it's a cold world out there.

### Creating Children

__Crossover Methods__

Taking a look back at the section answering `Question 2`, it's easy to draw the conclusion that all crossover methods will be detrimental to finding the optimal solution. In general, _order encoded_ problems are ill-suited for crossover methods, however, the TSP takes this to a new level by placing a restriction on the solution that effectively disqualifies most children created using _one-point_, _two-point_, or _uniform crossover_ : "All cities, except the origin city, must be visited exactly once." Because of this, crossover methods will not be used as primary reproduction methods.

__Mutation Methods__

Similar to crossover methods, most children created using _bounded_ or _arithmetic mutation_ will be disqualified. The one exception and selected primary method of reproduction is _order mutation_. _Order mutation_ already takes the unique ordering requirement into account and is the clear choice for this implementation.

__Addtional Considerations__
Unlike the process of selecting parents, the process of creating children can be implemented outside of the `Population` class as it doesn't require access to the data contained in the object. In fact, most of the methods only require access to one or two indidividuals which can be passed as parameters to a general, class-independent utility function. Nevertheless, these functions will still be included as utility functions in the `Population` class as they will undoubtedly be used exlcusively for manipulation of a `Population` object. 

_Notes_:
- Though order mutation is the primary reproduction method used, the crossover and other mutation methods may be implemented for testing and comparison purposes as time allows.
- Python referencing is great ... Sometimes it will, and sometimes it won't reference an object and it's your job to figure out which happened (•‿•).

### Populating Next Generation

This choice actually matters very little on the overall implementation. The only consideration here is the synergy of the stopping condition with the method selected. In general, the `generational` method will provide better solutions with a stopping condition that is reliant on overall population fitness (though it may skip over the best solution) while the `steady-state` method will tend to work better with a stopping condition based on a fixed number of generations. 

For the sake of giving a straight answer, this implementation will use the `steady-state` method as the primary method of populating new generations.

_Notes_:
- Steady state method replaces any individuals in the current population with new individuals(children) if a child's fitness score is equal or better than that individual's. This decision was made in an effort to avoid stagnation.

### Stopping the Algorithm

Similar to the method used to populate new generations, the choice of stopping condition does not have a meaningful impact on the implementation. Instead, the stopping condition should be decided based on the method for populating new generation for the reasons already mentioned.

Since the implementation uses the steady-state method, it's safe to set a fixed number of generations as the stopping condition.

_Notes_:
- Since the the initial population can satisfy the fitness threshold condition, the stopping condition is checked every generation.

### Additional Considerations & Results

Although most design decisions have been discussed already, the major remaining topic is the modularity of the code. You may have noticed that there are multiple interface-like functions such as `select_parents()` or `mutate_parents()`; there's a good reason for that : testing.

It's difficult to prove a given solution is the "optimal" one using genetic algorithms because of their pseudo-random nature. Running the algorithm multiple times and ending up with the same solution helps establish credibility, but without testing each and every possible outcome, the results are harder to prove correct. For this reason, I've __*attempted*__ to implement all of the various methods that a genetic algorithm can use to determine the optimal solution. By testing all of the different decision-making methods and (hopefully) ending up with the same solution, it's significantly more likely that the given solution is the best possible one.

In [5]:
def run_genetic_algorithm(seed:str, population_capacity:int, selection_method:Selection, crossover_method:Crossover,
                    mutation_method:Mutation, repopulation_method:Repopulation, stopping_condition:Stop,
                    **kwargs) -> tuple[Individual, int]:
    """Solve the TSP using a genetic algorithm.
    
    Args:
        seed : the seed used to acheive psuedo-randomization of initial population.
        population_capacity : the total amount of individuals allowed in the population.
        selection_method : the selection method to use.
                 Valid options are [Rank, Tournament, Elitism, Roulette_Wheel]
        crossover_method : the crossover method to use.
                         Valid options are [One_Point, Two_Point, Uniform]
        mutation_method : the mutation method to use.
                         Valid options are [Boundary, Arithmetic, Order]
        repopulation_method : the repopulation method to use.
                         Valid options are [Steady_State, Generational]
        stopping_condition : the stopping condition to use.
                        Valid options are [Generations, Threshold]
        **kwargs : additional named parameters dependent on desired methods.
    
    Returns:
        The a tuple containing "optimal" solution found by the algorithm and the combined population fitness.
    """ 
    
    # generate cities at random locations.
    cities = generate_random_cities(seed) if (
                kwargs.get("cities", None) == None) else generate_random_cities(seed, kwargs["cities"] )
    
    # create population.
    population = generate_initial_population(population_capacity, cities)
    
    # run algorithm until stopping condition met.
    while ((optimal_solution_found:= population.found_solution(stopping_condition, 
                kwargs["generations" if stopping_condition == Stop.Generations else "threshold"])) 
           == [False, None, None] ):
        
        # prevent fitness-based stopping condition from running forever.
        if ((stopping_condition == Stop.Threshold) and (population.current_generation == (kwargs["generations"] if (
            kwargs.get("generations", None) != None) else 10000)) ):
            return (None, None)
        
        # select parents based on desired selection method.
        selected_parents = population.select_parents(selection_method, kwargs["parents_needed"] if 
                                repopulation_method == Repopulation.Steady_State else population_capacity)
        
        # modify parents using desired crossover/mutation method.
        children = population.mutate_parents(mutation_method, selected_parents) if \
                mutation_method != None else population.crossover_parents(crossover_method, selected_parents)
        
        # modify population based on desired repopulation method.
        population.populate_next_generation(repopulation_method, children)
    
    # return "optimal" solution and combined population fitness.
    return (optimal_solution_found[1], optimal_solution_found[2])

### Testing

While most developers abhor testing, I find extremely helpful for finding careless mistakes and resolving logical inconsistencies (which happen a lot more than I'd like to admit). It's also kind of cool when things work as you'd expect them to on first try *chef's kiss*. Though some trivial verifications (like the result of completely random choice) are skipped in the interest of time and my happiness, there are still some sanity checks in here.

The code in this section is not required to complete the project nor for utilizing the implementation. Feel free to skip.

Also, I know I _could_ use the built-in unittest module but where's the fun in that? :)

In [6]:
def test_city_creation():
    results = True
    
    TEST_DATA = [
        {"x": 4, "y" : 3, "id": "My City A"},
        {"x": 6, "y" : 21, "id": "My City B"},
        {"x": 100, "y" : 1, "id": "My City C"},
        {"x": 15, "y" : 58, "id": "My City D"},
    ]
    
    for t in TEST_DATA:
        city = City(t["x"], t["y"], t["id"])
    
        # verify read operations work as expected.
        results = results                   \
                  and (city.id == t["id"])  \
                  and (city.x == t["x"])    \
                  and (city.y == t["y"]) 

    # verify attributes read-only.
    city = City(1, 2, "three")
    attrs = ["x", "y", "id"]
    to_set = [3, 2, "one"]
    for i in range(len(attrs)):
        try:
            setattr(city, attrs[i], to_set[i])
        except: # write failed.
            results = results and True
        else: # write successful.
            results = results and False
    return results
   
def test_city_distance_calcs():
    results = True
    
    TEST_DATA = [
        {"x": 4, "y" : 3, "id": "My City A"},
        {"x": 6, "y" : 21, "id": "My City B"},
        {"x": 100, "y" : 1, "id": "My City C"},
        {"x": 15, "y" : 58, "id": "My City D"},
    ]
    
    cities = {}
    for t in TEST_DATA:
        cities[t["id"]] = [City(t["x"], t["y"], t["id"]), 0]
        
    for curr_city in [cities[city_id][CITY] for city_id in cities]:
        curr_city.calculate_distance_to_other_cities(cities)
        
        # verify all distance calc (pythagorean theorem).
        for city_id in cities:
            
            # verify current city has no distance(None).
            if (cities[city_id][CITY] is curr_city):
                results = results and (curr_city.get_distance_to_city(city_id) == None)
                continue
            
            other_city = cities[city_id][CITY]
            exp_distance_x = abs(curr_city.x - other_city.x)
            exp_distance_y = abs(curr_city.y - other_city.y)
            exp_direct = round(sqrt(exp_distance_x**2+exp_distance_y**2), 3)
            
            results = results                                                 \
                      and (curr_city.get_distance_to_city(city_id) == (exp_direct))
            
    return results

def test_generate_random_cities():
    
    results = True
    
    # verify "random" generation is reproducible.
    seed = "test"
    TEST_DATA = [
        25,
        50,
        102,
        13,  
    ]
    
    for t in TEST_DATA:
        first_set = generate_random_cities(seed, t)
        second_set = generate_random_cities(seed, t)

        for f, s in zip(first_set, second_set):
            # check city ids (double check), x vals, and y vals
            results = results                                               \
                      and (f == s)                                          \
                      and (first_set[f][CITY].x == second_set[s][CITY].x)   \
                      and (first_set[f][CITY].y == second_set[s][CITY].y)   \
                      and (first_set[f][CITY].id == second_set[s][CITY].id) 
        
    return results

def test_calculate_fitness():
    results = True
    
    TEST_DATA = [
        {"x": 4, "y" : 3, "id": "My City A"},
        {"x": 6, "y" : 21, "id": "My City B"},
        {"x": 100, "y" : 1, "id": "My City C"},
        {"x": 15, "y" : 58, "id": "My City D"},
    ]
    
    cities = {}
    for i, t in enumerate(TEST_DATA):
        cities[t["id"]] = [City(t["x"], t["y"], t["id"]), i]
    
    for city in [cities[city_id][CITY] for city_id in cities]:
        city.calculate_distance_to_other_cities(cities)
        
    # calculate total travel distance of path.
    total_distance = sum([
        cities[TEST_DATA[i]["id"]][CITY].get_distance_to_city(TEST_DATA[i-1]["id"])
        for i in range(len(TEST_DATA))])
    
    # calculate expected fitness score.
    max_direct_distance = (len(cities) + 1)*round(sqrt((PLANE_DIMENSION_X**2)+(PLANE_DIMENSION_Y**2)), 3)
    exp_score = int( round( ((10**5) * max_direct_distance) - (total_distance * 1000), 3) )
    calc_score = calculate_fitness(cities)
    
    # verify calculated fitness score matches expected score and is an int.
    results = results                       \
              and (exp_score == calc_score) \
              and (type(calc_score) == int)
    
    # verify paths that don't visit all cities exactly once get fitness score of 0.
    INVALID_ORDERS = [ 2,   # assigned to My City C
                       7,   # out of bounds
                      -1,   # non-positive
                       1.3, # non-whole integer  
                     ]
    for io in INVALID_ORDERS:
        cities["My City B"][ORDER] = io 
        results = results                         \
              and (calculate_fitness(cities) == 0) 
    
    return results

def test_population_creation():
    results = True 
    
    TEST_DATA = [
        {"capacity": 30},
        {"capacity": 25},
        {"capacity": 100},
    ]
    
    for t in TEST_DATA:
        population = Population(t["capacity"])
    
        # verify read operations work as expected.
        results = results                                     \
                  and (population.capacity == t["capacity"])  \
                  and (population.current_generation == 0)  
        
    # verify attributes read-only.
    population = Population(12)
    attrs = ["capacity", "individuals", "current_generation"]
    to_set = [3, [None, None], 1]
    for i in range(len(attrs)):
        try:
            setattr(population, attrs[i], to_set[i])
        except: # write failed.
            results = results and True
        else: # write successful.
            results = results and False
    return results

def test_generate_initial_population():
    results = True
    
    TEST_DATA = [
        {"capacity": 30} ,
        {"capacity": 25},
        {"capacity": 100},
    ]
    
    seed = "test"
    for t in TEST_DATA:
        cities = generate_random_cities(seed)
        population = generate_initial_population(t["capacity"], cities)
        
        # verify population parameters as expected.
        results = results                                            \
                  and (population.capacity == t["capacity"])         \
                  and (population.current_generation == 0)           \
                  and (len(population.individuals) == t["capacity"])
        
        # verify no individuals with fitness scores of 0 in population.
        for individual in population.individuals:
            results = results                 \
                      and (calculate_fitness(individual) != 0)
    
    return results

def test_population_select_parents():
    
    results = True
    
    TEST_DATA = [
        {"capacity": 20, "parents_needed" : 3, "method" : Selection.Rank },
        {"capacity": 20, "parents_needed" : 3, "method" : Selection.Tournament },
        {"capacity": 20, "parents_needed" : 3, "method" : Selection.Elitism },
        {"capacity": 20, "parents_needed" : 3, "method" : Selection.Roulette_Wheel },
        {"capacity": 10, "parents_needed" : 16, "method" : Selection.Rank },
        {"capacity": 10, "parents_needed" : 18, "method" : Selection.Tournament },
        {"capacity": 10, "parents_needed" : 19, "method" : Selection.Elitism },
        {"capacity": 10, "parents_needed" : 20, "method" : Selection.Roulette_Wheel },
    ]
    
    cities = generate_random_cities("test")
    
    # verify the specified amount of parents are selected.
    for t in TEST_DATA:
        population = generate_initial_population(t["capacity"], cities)
        selected = population.select_parents(t["method"], t["parents_needed"])
        results = results                                    \
                  and (len(selected) == t["parents_needed"])
        
    
    return results

def test_population_mutate_parents():
    
    results = True
    
    TEST_DATA = [
        {"method" : Mutation.Order },
    ]
    
    cities = generate_random_cities("test")
    population = generate_initial_population(30, cities)
    selected = population.select_parents(Selection.Rank, 5)
    
    # verify mutated children have valid fitness scores and are not referencing the the objects stored in 
    # the current population.
    for t in TEST_DATA:
        mutated_children = population.mutate_parents(t["method"], selected)
        
        for i, ind in enumerate(mutated_children):
            results = results                                            \
                      and (calculate_fitness(ind) != 0)                  \
                      and not (ind is selected[i])
    
    return results

def test_population_populate_next_generation():
    results = True
    
    TEST_DATA = [
        {"method" : Repopulation.Steady_State, "capacity" : 10, "to_replace": 3},
        {"method" : Repopulation.Steady_State, "capacity" : 20, "to_replace": 10},
        {"method" : Repopulation.Steady_State, "capacity" : 10, "to_replace": 30},
        {"method" : Repopulation.Steady_State, "capacity" : 1000, "to_replace": 30},
        {"method" : Repopulation.Steady_State, "capacity" : 1000, "to_replace": 1000},
        {"method" : Repopulation.Generational, "capacity" : 10, "to_replace": 3},
        {"method" : Repopulation.Generational, "capacity" : 20, "to_replace": 10},
        {"method" : Repopulation.Generational, "capacity" : 10, "to_replace": 30},
        {"method" : Repopulation.Generational, "capacity" : 1000, "to_replace": 30},
        {"method" : Repopulation.Generational, "capacity" : 1000, "to_replace": 1000},
    ]
    
    for t in TEST_DATA:
        cities = generate_random_cities("test")
        population = generate_initial_population(t["capacity"], cities)
        replacements = population.mutate_parents(Mutation.Order, population.select_parents(Selection.Rank, t["to_replace"]))
        old_pop = population.individuals
        population.populate_next_generation(t["method"], replacements)
        new_pop = population.individuals
        
        
        if (t["method"] == Repopulation.Steady_State):
            
            # get fitness scores in order unsorted order.
            prev_pop_fs = [calculate_fitness(individual) for individual in old_pop]
            rpm_fs = [calculate_fitness(individual) for individual in replacements]
            new_pop_fs = [calculate_fitness(individual) for individual in population.individuals]
            
            
            groups = [
                {"fs" : rpm_fs, "individuals" : replacements },
                {"fs" : prev_pop_fs, "individuals" : old_pop },
            ]
            prev_pop_matches = [matching_fs for matching_fs in list(set(prev_pop_fs) & set(new_pop_fs))]
            rpm_matches = [matching_fs for matching_fs in list(set(rpm_fs) & set(new_pop_fs))]
            
            # find all individuals with matching fs in replacements and previous population.
            failures_allowed = len([matching_fs for matching_fs in list(set(prev_pop_matches) & set(rpm_matches))])
            
            # find all matching fitness scores from prev pop and replacements (can overlap).
            failures = 0
            for group in groups:
                matches = [matching_fs for matching_fs in list(set(group["fs"]) & set(new_pop_fs))]
                
                # verify individuals with matching fitness are in new generation.
                for match in matches:
                    individual = group["individuals"][group["fs"].index(match)]
                    new_pop_individual = new_pop[new_pop_fs.index(match)]
                    
                    # ignore failures stemming from individuals with the same fs. 
                    # Tracking these failures would add unneccesary complications to the test.
                    failures += 0 if ((individual is new_pop_individual) or \
                            (calculate_fitness(individual) == calculate_fitness(new_pop_individual))) else 1
            results = results and (failures <= failures_allowed)
        else:
            # verify no individuals from old population in new population.
            results = results                                           \
                      and (len([individual for individual in new_pop if individual in old_pop]) == 0)
        # verify population within capacity.
        results = results                                \
                and (len(population.individuals) == population.capacity)
            
    return results

def test_population_stopping_condition():
    
    results = True
    
    cities = generate_random_cities("test")
    population = generate_initial_population(25, cities)
    cities = generate_random_cities("test")
    replacements = population.mutate_parents(Mutation.Order, population.select_parents(Selection.Rank, 5))
    

    max_generations = 3
    for _ in range(max_generations):
        
        # verify stopping condition not detected.
        results = results                                                                                  \
                  and (population.found_solution(Stop.Generations, max_generations) == [False, None, None])
        
        # repopulate.
        replacements = population.mutate_parents(Mutation.Order, population.select_parents(Selection.Rank, 5))
        population.populate_next_generation(Repopulation.Steady_State, replacements)
    
    # verify stopping condition detected and fittest individual returned.
    fs = calculate_fitness_multi(population.individuals)
    total_fs = sum([entry["score"] for entry in fs])
    fittest_ind = population.individuals[fs[-1]["individual_index"]]
    
    # verify stopping condition detected for generational method and fittest individual returned.
    results = results                                                                                        \
          and (population.found_solution(Stop.Generations, max_generations) == [True, fittest_ind, total_fs])
    
    # verify stopping condition detected for fitness threshold method and fittest individual returned.
    results = results                                                                                \
          and (population.found_solution(Stop.Threshold, total_fs+1) == [False, None, None])         \
          and (population.found_solution(Stop.Threshold, total_fs) == [True, fittest_ind, total_fs])
        
    return results
    

def run_tests():

    tests = [test_city_creation, test_city_distance_calcs, test_generate_random_cities, test_calculate_fitness,
             test_population_creation, test_generate_initial_population, test_population_select_parents, 
             test_population_mutate_parents, test_population_populate_next_generation, test_population_stopping_condition]

    print(f"Running {len(tests)} Tests.")

    failed = {"number" : 0, "names": []}
    for test in tests:
        if (not test()):
            failed["number"] += 1
            failed["names"].append(test.__name__)

    print(f"Test Results:\n\
    Passed: {len(tests) - failed['number']}\n\
    Failed: {failed['number']}\n\t\
    {failed['names'] if len(failed['names']) > 0 else '' }")


run_tests()

Running 10 Tests.
Test Results:
    Passed: 10
    Failed: 0
	    


### Finally...

25 hours spread across 4 days, 42 Stack Overflow searches, 72 GeeksforGeeks articles, and 162 stupid bugs later, and we can finally run the algorith. Let's see what we get.

In [7]:
def get_solution_information(solution) -> tuple[float, str]:
    """Gets pertinent solution information.
    
    Args:
        solution : ... the solution.
    Returns :
        ............ the pertinent solution information. I need another cup of coffee.
    """
    
    # get total path distance and travel order.
    distance = calculate_path_distance(solution)
    order = "Travel Order:\n"
    travel_order = [-6 for _ in range(len(solution))]
    # determine actual travel order of solution.
    for city_id in solution:

        entry = solution[city_id]

        # set city in correct travel order.
        travel_order[entry[ORDER]] = entry[CITY]

    for city in travel_order:
        order += f"\tCity: {city.id}, coords (x: {city.x}, y: {city.y})\n"

    return distance, order

solution, total_fs = run_genetic_algorithm(seed="please work",
                            population_capacity= 25,
                            selection_method = Selection.Rank,
                            crossover_method = None,
                            mutation_method = Mutation.Order,
                            repopulation_method = Repopulation.Steady_State,
                            stopping_condition = Stop.Generations,
                            parents_needed=15,
                            generations=30) 

solution_info = get_solution_information(solution)
print(f"Path distance : {solution_info[0]} km.\nTotal Population Fitness: {total_fs}.\n{solution_info[1]}")

Path distance : 1377.748 km.
Total Population Fitness: 18349704721.
Travel Order:
	City: k0, coords (x: 156, y: 81)
	City: p0, coords (x: 171, y: 33)
	City: y0, coords (x: 119, y: 153)
	City: e0, coords (x: 184, y: 145)
	City: f0, coords (x: 181, y: 155)
	City: v0, coords (x: 182, y: 76)
	City: i0, coords (x: 185, y: 80)
	City: q0, coords (x: 100, y: 110)
	City: b0, coords (x: 57, y: 109)
	City: h0, coords (x: 9, y: 161)
	City: d0, coords (x: 70, y: 150)
	City: l0, coords (x: 80, y: 135)
	City: s0, coords (x: 62, y: 122)
	City: c0, coords (x: 55, y: 38)
	City: w0, coords (x: 55, y: 44)
	City: n0, coords (x: 7, y: 39)
	City: g0, coords (x: 62, y: 28)
	City: o0, coords (x: 120, y: 29)
	City: m0, coords (x: 84, y: 60)
	City: t0, coords (x: 96, y: 108)
	City: a0, coords (x: 98, y: 68)
	City: u0, coords (x: 120, y: 27)
	City: r0, coords (x: 55, y: 157)
	City: j0, coords (x: 107, y: 164)
	City: x0, coords (x: 130, y: 131)



Well, that was anticlimactic. 

Let's try some larger numbers. This might take some time...


In [8]:
import statistics

REPS = 10
CAPACITY = 200
GENERATIONS = 1000
MUTATION_RATE = 50

distances = []
paths = []
population_fs = []
for rep in range(REPS):
    solution, total_fs = run_genetic_algorithm(seed="please work",
                                population_capacity= CAPACITY,
                                selection_method = Selection.Rank,
                                crossover_method = None,
                                mutation_method = Mutation.Order,
                                repopulation_method = Repopulation.Steady_State,
                                stopping_condition = Stop.Generations,
                                parents_needed=MUTATION_RATE,
                                generations=GENERATIONS) 
    solution_info = get_solution_information(solution)
    distances.append(solution_info[0])
    paths.append(solution_info[1])
    population_fs.append(total_fs)
    
print(f"Shortest Path: {min(distances)} km.\
      \nLongest Path : {max(distances)} km.\
      \nAverage Distance : {round(statistics.mean(distances), 3)} km.\
      \nStandard Deviation(Distance): {round(statistics.pstdev(distances), 3)} km.\
      \nAverage Population Fitness : {int(statistics.mean(population_fs))}.\n\n")

for i in range(len(distances)):
    print(f"Path distance : {distances[i]} km.\n{paths[i]}\n")
    

Shortest Path: 809.18 km.      
Longest Path : 935.013 km.      
Average Distance : 857.924 km.      
Standard Deviation(Distance): 41.796 km.      
Average Population Fitness : 146905143908.


Path distance : 809.262 km.
Travel Order:
	City: o0, coords (x: 120, y: 29)
	City: u0, coords (x: 120, y: 27)
	City: p0, coords (x: 171, y: 33)
	City: k0, coords (x: 156, y: 81)
	City: v0, coords (x: 182, y: 76)
	City: i0, coords (x: 185, y: 80)
	City: e0, coords (x: 184, y: 145)
	City: f0, coords (x: 181, y: 155)
	City: x0, coords (x: 130, y: 131)
	City: y0, coords (x: 119, y: 153)
	City: j0, coords (x: 107, y: 164)
	City: d0, coords (x: 70, y: 150)
	City: r0, coords (x: 55, y: 157)
	City: h0, coords (x: 9, y: 161)
	City: b0, coords (x: 57, y: 109)
	City: s0, coords (x: 62, y: 122)
	City: l0, coords (x: 80, y: 135)
	City: q0, coords (x: 100, y: 110)
	City: t0, coords (x: 96, y: 108)
	City: a0, coords (x: 98, y: 68)
	City: m0, coords (x: 84, y: 60)
	City: w0, coords (x: 55, y: 44)
	City: n0, coo

and, Voilà; what a difference. We've got some relatively precise results here and a lot better of an answer. But, I'm curious as to what the practical differences are using each method.

Let's go bigger! This will take definitely take a while. There will probably be another Transformers or Fast and Furious movie that's released before this program finishes; at least you can enjoy that...

In [9]:
from time import perf_counter as stopwatch
        
SEED = "please work"
CAPACITY = 1000
GENERATIONS = 1000 
MAX_DISTANCE = 26 * sqrt((PLANE_DIMENSION_X**2)+(PLANE_DIMENSION_Y**2))

# calculate threshold based on primary method results.
population_fs = []
for rep in range(10):
    _, total_fs = run_genetic_algorithm(seed=SEED,
                                    population_capacity= CAPACITY,
                                    selection_method = Selection.Rank,
                                    crossover_method = None,
                                    mutation_method = Mutation.Order,
                                    repopulation_method = Repopulation.Steady_State,
                                    stopping_condition = Stop.Generations,
                                    parents_needed=25,
                                    generations=1000,
                                    threshold=None)  
    population_fs.append(total_fs)
THRESHOLD = int(statistics.mean(population_fs))
    

def get_str_selection(method):
    if method == Selection.Rank:
        return "Rank"
    elif method == Selection.Tournament:
        return "Tournament"
    elif method == Selection.Elitism:
        return "Elitism"
    elif method == Selection.Roulette_Wheel:
        return "Roulette-Wheel"

def get_str_mutation(method):
    if method == Mutation.Aritmetic:
        return "Arithmetic"
    elif method == Mutation.Boundary:
        return "Boundary"
    elif method == Mutation.Order:
        return "Order"

def get_str_repopulation(method):
    if method == Repopulation.Steady_State:
        return "Steady-State"
    elif method == Repopulation.Generational:
        return "Generational"

def get_str_stop(method):
    if method == Stop.Generations:
        return "Fixed Generations"
    if method == Stop.Threshold:
        return "Fitness Threshold"

TEST_DATA = [
    # Rank Selection
    {"selection_method": Selection.Rank, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Steady_State, 
     "stopping_condition": Stop.Generations, "mutation_rate": 25, "generations": GENERATIONS, "threshold": None},
    {"selection_method": Selection.Rank, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Steady_State, 
     "stopping_condition": Stop.Generations, "mutation_rate": 25, "generations": int(GENERATIONS*2), "threshold": None},
    {"selection_method": Selection.Rank, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Generational, 
     "stopping_condition": Stop.Generations, "mutation_rate": CAPACITY, "generations": GENERATIONS, "threshold": None},
    {"selection_method": Selection.Rank, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Generational, 
     "stopping_condition": Stop.Generations, "mutation_rate": CAPACITY, "generations": int(GENERATIONS*2), "threshold": None},
    {"selection_method": Selection.Rank, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Steady_State, 
     "stopping_condition": Stop.Threshold, "mutation_rate": 25, "generations": int(GENERATIONS*10), "threshold": THRESHOLD},
    {"selection_method": Selection.Rank, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Generational, 
     "stopping_condition": Stop.Threshold, "mutation_rate": CAPACITY, "generations": int(GENERATIONS*10), "threshold": int(THRESHOLD/(3/4))}, 
    # Tournament selection
    {"selection_method": Selection.Tournament, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Steady_State, 
     "stopping_condition": Stop.Generations, "mutation_rate": 25, "generations": GENERATIONS, "threshold": None},
    {"selection_method": Selection.Tournament, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Steady_State, 
     "stopping_condition": Stop.Generations, "mutation_rate": 25, "generations": int(GENERATIONS*2), "threshold": None},
    {"selection_method": Selection.Tournament, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Generational, 
     "stopping_condition": Stop.Generations, "mutation_rate": CAPACITY, "generations": GENERATIONS, "threshold": None},
    {"selection_method": Selection.Tournament, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Generational, 
     "stopping_condition": Stop.Generations, "mutation_rate": CAPACITY, "generations": int(GENERATIONS*2), "threshold": None},
    {"selection_method": Selection.Tournament, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Steady_State, 
     "stopping_condition": Stop.Threshold, "mutation_rate": 25, "generations": int(GENERATIONS*10), "threshold": THRESHOLD},
    {"selection_method": Selection.Tournament, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Generational, 
     "stopping_condition": Stop.Threshold, "mutation_rate": CAPACITY, "generations": int(GENERATIONS*10), "threshold": int(THRESHOLD/(3/4))},
    # Elitism selection
    {"selection_method": Selection.Elitism, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Steady_State, 
     "stopping_condition": Stop.Generations, "mutation_rate": 25, "generations": GENERATIONS, "threshold": None},
    {"selection_method": Selection.Elitism, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Steady_State, 
     "stopping_condition": Stop.Generations, "mutation_rate": 25, "generations": int(GENERATIONS*2), "threshold": None},
    {"selection_method": Selection.Elitism, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Generational, 
     "stopping_condition": Stop.Generations, "mutation_rate": CAPACITY, "generations": GENERATIONS, "threshold": None},
    {"selection_method": Selection.Elitism, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Generational, 
     "stopping_condition": Stop.Generations, "mutation_rate": CAPACITY, "generations": int(GENERATIONS*2), "threshold": None},
    {"selection_method": Selection.Elitism, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Steady_State, 
     "stopping_condition": Stop.Threshold, "mutation_rate": 25, "generations": int(GENERATIONS*10), "threshold": THRESHOLD},
    {"selection_method": Selection.Elitism, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Generational, 
     "stopping_condition": Stop.Threshold, "mutation_rate": CAPACITY, "generations": int(GENERATIONS*10), "threshold": int(THRESHOLD/(3/4))},
    # Roulette-Wheel selection
    {"selection_method": Selection.Roulette_Wheel, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Steady_State, 
     "stopping_condition": Stop.Generations, "mutation_rate": 25, "generations": GENERATIONS, "threshold": None},
    {"selection_method": Selection.Roulette_Wheel, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Steady_State, 
     "stopping_condition": Stop.Generations, "mutation_rate": 25, "generations": int(GENERATIONS*2), "threshold": None},
    {"selection_method": Selection.Roulette_Wheel, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Generational, 
     "stopping_condition": Stop.Generations, "mutation_rate": CAPACITY, "generations": GENERATIONS, "threshold": None},
    {"selection_method": Selection.Roulette_Wheel, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Generational, 
     "stopping_condition": Stop.Generations, "mutation_rate": CAPACITY, "generations": int(GENERATIONS*2), "threshold": None},
    {"selection_method": Selection.Roulette_Wheel, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Steady_State, 
     "stopping_condition": Stop.Threshold, "mutation_rate": 25, "generations": int(GENERATIONS*10), "threshold": THRESHOLD},
    {"selection_method": Selection.Roulette_Wheel, "crossover_method" : None,
     "mutation_method" : Mutation.Order, "repopulation_method": Repopulation.Generational, 
     "stopping_condition": Stop.Threshold, "mutation_rate": CAPACITY, "generations": int(GENERATIONS*10), "threshold": int(THRESHOLD/(3/4))},
]

all_paths = []
best_time = [float(inf), None]
best_distance = [float(inf), None]
worst_time = [float(-inf), None]
worst_distance = [float(-inf), None]
best_path = None
worst_path = None
results = ""

distances = []
for t in TEST_DATA:
    start = stopwatch()
    curr_config = f"selection : {get_str_selection(t['selection_method'])}, repopulation: {get_str_repopulation(t['repopulation_method'])}, stopping condition: {get_str_stop(t['stopping_condition'])}."
    # run each configuration
    solution, _ = run_genetic_algorithm(seed=SEED,
                            population_capacity= CAPACITY,
                            selection_method = t["selection_method"],
                            crossover_method = t["crossover_method"],
                            mutation_method = t["mutation_method"],
                            repopulation_method = t["repopulation_method"],
                            stopping_condition = t["stopping_condition"],
                            parents_needed=t["mutation_rate"],
                            generations=t["generations"],
                            threshold=t["threshold"]) 
    
    # change output if solution not found based on constraints.
    total_time = round(stopwatch() - start, 3)
    if (solution != None):
        solution_info = get_solution_information(solution)
        distances.append(solution_info[0])
        
        # get best/worst distances and associated configs, paths.
        if (max(worst_distance[0], solution_info[0]) == solution_info[0]):
            worst_distance[0] = solution_info[0]
            worst_distance[1] = f"Config - {curr_config}"
            worst_path = solution_info[1]

        if (min(best_distance[0], solution_info[0]) == solution_info[0]):
            best_distance[0] = solution_info[0]
            best_distance[1] = f"Config - {curr_config}"
            best_path = solution_info[1]

        # get best/worst times and associated configs.
        if (max(worst_time[0], total_time) == total_time):
            worst_time[0] = total_time
            worst_time[1] = f"Config - {curr_config}"

        if (min(best_time[0], total_time) == total_time):
            best_time[0] = total_time
            best_time[1] = f"Config - {curr_config}"


        # save all distances, paths, and run-time for each configuration
        results += f"Config - {curr_config} \
          \n\tPopulation Capacity: {CAPACITY}. \
          \n\tMutation Rate: {t['mutation_rate']}. \
          \n\tCalculation Time: {total_time} seconds.\
          \n\tDistance: {solution_info[0]} km.\n\n"
    else:
        results += f"Config - {curr_config} \
          \n\tPopulation Capacity: {CAPACITY}. \
          \n\tMutation Rate: {t['mutation_rate']}. \
          \n\tCalculation Time: {total_time} seconds.\
          \n\tCould not find valid solution within reasonable time.\n\n"
    
results += f"Average Distance : {round(statistics.mean(distances), 3)} km.\
             \nStandard Deviation(Distance): {round(statistics.pstdev(distances), 3)} km.\
             \nBest Time: {best_time[0]} seconds.\
             \n\tConfig: {best_time[1]}.\
             \nWorst Time: {worst_time[0]} seconds.\
             \n\tConfig: {worst_time[1]}.\
             \nBest Distance: {best_distance[0]} km.\
             \n\tConfig: {best_distance[1]}.\
             \nPath: \n{best_path}.\
             \nWorst Distance: {worst_distance[0]} km.\
             \n\tConfig: {worst_distance[1]}.\
             \nPath: \n{worst_path}."



print(results)

Config - selection : Rank, repopulation: Steady-State, stopping condition: Fixed Generations.           
	Population Capacity: 1000.           
	Mutation Rate: 25.           
	Calculation Time: 40.834 seconds.          
	Distance: 1090.409 km.

Config - selection : Rank, repopulation: Steady-State, stopping condition: Fixed Generations.           
	Population Capacity: 1000.           
	Mutation Rate: 25.           
	Calculation Time: 82.795 seconds.          
	Distance: 937.305 km.

Config - selection : Rank, repopulation: Generational, stopping condition: Fixed Generations.           
	Population Capacity: 1000.           
	Mutation Rate: 1000.           
	Calculation Time: 386.427 seconds.          
	Distance: 1382.863 km.

Config - selection : Rank, repopulation: Generational, stopping condition: Fixed Generations.           
	Population Capacity: 1000.           
	Mutation Rate: 1000.           
	Calculation Time: 776.372 seconds.          
	Distance: 1448.309 km.

Config - select

### Final Thoughts & Retrospective

And there we have it; finished at last. What a ride it's been.

It looks like this relatively "small" test was useful for benchmarking. Here, we can clearly see that the fitness threshold stopping condition will either make the algorithm or break it. This observation alludes back to the sections describing the repopulation method and stopping condition.

Lastly, any functionality not already implemented is most likely not going to be – and there are some good reasons for that. One, the functionality in question is not a good fit for the problem space and will only increase code complexity. Two – and this is very important – is because I don't want to ¯\_(ツ)_/¯.

In all seriousness, I've accomplished my goal to learn as much about the genetic algorithm as possible through practical application and the project goals described in the opening section. Could the application be more efficient? Of course. Are there places where the code can be refactored? Probably. Nevertheless, I believe the original goals of the project have been met and that makes this as good a stopping point as any.

_Notes_:
- An extra constraint was added to the genetic algorithm for runs with the fitness threshold stopping condition to prevent the algorithm from running for ridiculously long times. Without this constraint the algorithm has run for a record of at least 10 hours for one configuration.
